In [1]:
# Requests for handling HTTP get and other requests
import requests
import pandas as pd
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
# 
from bs4 import BeautifulSoup
import time
import re

In [21]:
url = "https://www.ss.com/lv/real-estate/flats/riga/"
baseurl = "https://www.ss.com"
postfix = 'sell/'

In [4]:
def getUrlList(url, prefix='https://www.ss.com', postfix='sell/', tag='a', class_='a_category'):
    """
    Returns a list of all region urls including citi etc
    """
    req = requests.get(url)
    if req.status_code != 200:
        print(f'Unexpected status code {req.status_code}. Stopping parse')
        return [] #return early and often principle
    soup = BeautifulSoup(req.text, 'lxml') # could skip soup variable as well but keeping for readability
    return [ prefix + el['href'] + postfix for el in soup.find_all(tag, class_) ]
    # What else could we pass as argument? How could our return fail?

In [10]:
def getAllRegionUrls(regionURL):
    """
    Returns a list of all urls for a specific region/rajons
    """
    print("Processing", regionURL)
    req = requests.get(regionURL)
    if req.status_code != 200:
        return []
    csoup = BeautifulSoup(req.text, 'lxml')
    pageanchors = csoup.find_all('a', {'name':'nav_id'})
    allRegionUrls = [regionURL] # base scenario we only get a list with the starting url
    if len(pageanchors) == 0:
        return allRegionUrls
    href = pageanchors[0].get('href')
    if not href:
        return allRegionUrls
    pagecount = int(href.split(".")[0].split("page")[-1])
    extraurls = [regionURL+"page"+str(n)+".html" for n in range(2, pagecount+1)]
    allRegionUrls += extraurls
    return allRegionUrls

In [5]:
def getRowData(row):
    """
    Given a table row
    Returns all text for elements past 2 plus first anchor link found"""
    return [el.text for el in row.find_all('td')[2:]] + [baseurl + row.find('a')['href']]

In [6]:
def getDFfromUrl(url, region = None, typeFilter = "Pārdod"):
    print(f'Going to gather data from URL:{url}')
    req = requests.get(url)
    if req.status_code != 200:
        print(f'Unexpected status code {req.status_code}. Stopping parse')
        return pd.DataFrame({}) #return early and often principle
    soup = BeautifulSoup(req.text, 'lxml') # could skip soup variable as well but keeping for readability
    

    
    headline = soup.find('tr', id = "head_line")
    cindex = [el.text for el in headline.find_all('td')]
    cindex[0] = cindex[0].split()[0]
    cindex += ['URL'] #TODO add argument for this
    cindex += ['Region']
    
    # TODO move it somewhere else
    if len([el for el in soup.find_all('option') if typeFilter in el.text]) == 0:
        print("Oops nothing for ",typeFilter)
        return pd.DataFrame({}, columns=cindex)
    
    rows = soup.find_all('tr',id = re.compile(r'tr_[0-9]+'))
    rowsdata = [getRowData(el) for el in rows]
    # finally we add the region if we did not have one
    if region == None:
        region = url.split("/")[-3]
    rowsdata = [el + [region] for el in rowsdata]
    return pd.DataFrame(rowsdata, columns=cindex)

In [18]:
# with this recipe we can append a big list of dataframes into one
def getDFfromUrlList(urlist):
    """
    Returns a dataframe combined from returns of individual urls
    """
    dflist = []
    for ur in urlist:
        dflist.append(getDFfromUrl(ur))
        time.sleep(0.3)
    return pd.concat(dflist)

In [14]:
def getFullUrlList(url):
    """
    Get a list of ALL urls from all regions including subpages
    """
    allregions = getUrlList(url)
    allregions = allregions[:-2] # for sales we do not need the last two visi and citi
    biglist = []
    for regurl in allregions:
        biglist += getAllRegionUrls(regurl)
        time.sleep(0.2)
    return biglist

In [15]:
biglist = getFullUrlList(url)

Processing https://www.ss.com/lv/real-estate/flats/riga/centre/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/agenskalns/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/aplokciems/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/beberbeki/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/bergi/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/bierini/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/bolderaya/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/breksi/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/bukulti/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/bulli/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/chiekurkalns/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/darzciems/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/daugavgriva/sell/
Processing https://www.ss.com/lv/real-estate/flats/riga/dreilini/sell/
Proc

In [16]:
biglist[:5]

['https://www.ss.com/lv/real-estate/flats/riga/centre/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html',
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html',
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page4.html',
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page5.html']

In [17]:
biglist[-10:]

['https://www.ss.com/lv/real-estate/flats/riga/voleri/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/zakusala/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/zasulauks/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page2.html',
 'https://www.ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page3.html',
 'https://www.ss.com/lv/real-estate/flats/riga/ziepniekkalns/sell/page4.html',
 'https://www.ss.com/lv/real-estate/flats/riga/zolitude/sell/',
 'https://www.ss.com/lv/real-estate/flats/riga/zolitude/sell/page2.html',
 'https://www.ss.com/lv/real-estate/flats/riga/vef/sell/']

In [22]:
df = getDFfromUrlList(biglist)

Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page4.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page5.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page6.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page7.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page8.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page9.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page10.html
Going to gather data from URL

Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page2.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page3.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page4.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page5.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page6.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page7.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page8.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/purvciems/sell/page9.html
Going to gather data from URL:https://www.ss.com/lv/real-estate/flats/riga/shampeteris-pleskodale/sel

In [23]:
df.shape

(2669, 10)

In [24]:
df.to_csv("apartments.csv")

In [25]:
df2 = pd.read_csv("apartments.csv")
df2.head()

,Unnamed: 0,Sludinājumi,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena,URL,Region
0,0,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,96,5/6,P. kara,"3,385 €","325,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
1,1,"Pārdod 3-istabu dzīvokli Rīgas klusajā centrā,...",Ausekļa 1,3,74,3/5,P. kara,"2,297 €","170,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
2,2,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,95,2/6,P. kara,"2,495 €","237,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
3,3,"Samazināta cena, iegūsti savu Laimi. \r\n\r\nP...",Valdemāra 123,3,102,6/6,P. kara,"1,372 €","139,900 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
4,4,В доме всего 6 квартир. Печное отопление (комм...,Pērnavas 43,3,54,3/6,P. kara,950 €,"51,300 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre


In [29]:
df2.describe()

,Unnamed: 0,m2
count,2669.000000,2669.000000
mean,13.739228,65.579618
std,8.856437,36.874368
min,0.000000,12.000000
25%,6.000000,44.000000
50%,13.000000,57.000000
75%,21.000000,78.000000
max,40.000000,427.000000


In [28]:
regions = df2.groupby(["Region"]).describe()
regions

Unnamed: 0                                          \
                             count       mean        std  min    25%   50%   
Region                                                                       
agenskalns                   146.0  14.143836   8.525006  0.0   7.00  14.0   
aplokciems                     9.0   4.000000   2.738613  0.0   2.00   4.0   
bergi                          4.0   1.500000   1.290994  0.0   0.75   1.5   
bierini                       11.0   5.000000   3.316625  0.0   2.50   5.0   
bolderaya                     34.0  16.500000   9.958246  0.0   8.25  16.5   
breksi                         1.0   0.000000        NaN  0.0   0.00   0.0   
bukulti                        1.0   0.000000        NaN  0.0   0.00   0.0   
bulli                          1.0   0.000000        NaN  0.0   0.00   0.0   
centre                       757.0  14.393659   8.693235  0.0   7.00  14.0   
chiekurkalns                  27.0  13.000000   7.937254  0.0   6.50  13.0   
darzciems                     32.0  15.500000   9.380832  0.0   7.75  15.5   
daugavgriva                   20.0   9.500000   5.916080  0.0   4.75   9.5   
dreilini                       2.0   0.500000   0.707107  0.0   0.25   0.5   
dzeguzhkalns                  28.0  13.500000   8.225975  0.0   6.75  13.5   
grizinkalns                   17.0   8.000000   5.049752  0.0   4.00   8.0   
ilguciems                     75.0  13.000000   8.581879  0.0   6.00  12.0   
imanta                       152.0  14.315789   8.774150  0.0   7.00  14.0   
jaunciems                      3.0   1.000000   1.000000  0.0   0.50   1.0   
jaunmilgravis                  3.0   1.000000   1.000000  0.0   0.50   1.0   
katlakalns                     3.0   1.000000   1.000000  0.0   0.50   1.0   
kengarags                    163.0  13.822086   8.707823  0.0   6.00  13.0   
kipsala                        8.0   3.500000   2.449490  0.0   1.75   3.5   
kleisti                        2.0   0.500000   0.707107  0.0   0.25   0.5   
kliversala                    11.0   5.000000   3.316625  0.0   2.50   5.0   
krasta-st-area                21.0  10.000000   6.204837  0.0   5.00  10.0   
kundzinsala                    1.0   0.000000        NaN  0.0   0.00   0.0   
mangali                       15.0   7.000000   4.472136  0.0   3.50   7.0   
mangalsala                     1.0   0.000000        NaN  0.0   0.00   0.0   
maskavas-priekshpilseta       35.0  17.000000  10.246951  0.0   8.50  17.0   
mezhapark                     41.0  20.000000  11.979149  0.0  10.00  20.0   
mezhciems                     78.0  13.115385   8.434593  0.0   6.00  12.5   
plyavnieki                   131.0  13.702290   8.774697  0.0   6.00  13.0   
purvciems                    244.0  14.286885   8.760630  0.0   7.00  14.0   
sarkandaugava                 58.0  14.017241   8.469743  0.0   7.00  14.0   
shampeteris-pleskodale        36.0  17.500000  10.535654  0.0   8.75  17.5   
shkirotava                     9.0   4.000000   2.738613  0.0   2.00   4.0   
teika                         82.0  13.426829   8.343003  0.0   6.25  13.0   
tornjakalns                   18.0   8.500000   5.338539  0.0   4.25   8.5   
vecaki                         1.0   0.000000        NaN  0.0   0.00   0.0   
vecmilgravis                  86.0  13.895349   8.422699  0.0   7.00  14.0   
vecriga                       60.0  14.500000   8.728484  0.0   7.00  14.5   
vef                            2.0   0.500000   0.707107  0.0   0.25   0.5   
voleri                         2.0   0.500000   0.707107  0.0   0.25   0.5   
yugla                         67.0  13.298507   9.005074  0.0   5.00  13.0   
zasulauks                      4.0   1.500000   1.290994  0.0   0.75   1.5   
ziepniekkalns                115.0  13.956522   8.462913  0.0   7.00  14.0   
zolitude                      52.0  12.807692   8.087847  0.0   6.00  12.5   

                                         m2                                \
                           75%   max  count   

In [31]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2669 entries, 0 to 2668
Data columns (total 11 columns):
Unnamed: 0     2669 non-null int64
Sludinājumi    2669 non-null object
Iela           2669 non-null object
Ist.           2669 non-null object
m2             2669 non-null int64
Stāvs          2669 non-null object
Sērija         2669 non-null object
Cena, m2       2669 non-null object
Cena           2669 non-null object
URL            2669 non-null object
Region         2669 non-null object
dtypes: int64(2), object(9)
memory usage: 229.4+ KB


In [34]:
df2.columns

Index(['Sludinājumi', 'Iela', 'Ist.', 'm2', 'Stāvs', 'Sērija', 'Cena, m2',
       'Cena', 'URL', 'Region'],
      dtype='object')

In [33]:
del df2['Unnamed: 0']

In [35]:
df2.head()

,Sludinājumi,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena,URL,Region
0,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,96,5/6,P. kara,"3,385 €","325,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
1,"Pārdod 3-istabu dzīvokli Rīgas klusajā centrā,...",Ausekļa 1,3,74,3/5,P. kara,"2,297 €","170,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
2,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,95,2/6,P. kara,"2,495 €","237,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
3,"Samazināta cena, iegūsti savu Laimi. \r\n\r\nP...",Valdemāra 123,3,102,6/6,P. kara,"1,372 €","139,900 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre
4,В доме всего 6 квартир. Печное отопление (комм...,Pērnavas 43,3,54,3/6,P. kara,950 €,"51,300 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre


In [41]:
df2["Price"] = df2["Cena"].str.extract('([0-9,]+)', expand=True)
df2.head()

,Sludinājumi,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena,URL,Region,Price
0,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,96,5/6,P. kara,"3,385 €","325,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,"325,000"
1,"Pārdod 3-istabu dzīvokli Rīgas klusajā centrā,...",Ausekļa 1,3,74,3/5,P. kara,"2,297 €","170,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,"170,000"
2,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,95,2/6,P. kara,"2,495 €","237,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,"237,000"
3,"Samazināta cena, iegūsti savu Laimi. \r\n\r\nP...",Valdemāra 123,3,102,6/6,P. kara,"1,372 €","139,900 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,"139,900"
4,В доме всего 6 квартир. Печное отопление (комм...,Pērnavas 43,3,54,3/6,P. kara,950 €,"51,300 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,"51,300"


In [42]:
df2["Price"] = df2["Price"].apply(lambda el: int(el.replace(",","")))
df2.head()

,Sludinājumi,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena,URL,Region,Price
0,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,96,5/6,P. kara,"3,385 €","325,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,325000
1,"Pārdod 3-istabu dzīvokli Rīgas klusajā centrā,...",Ausekļa 1,3,74,3/5,P. kara,"2,297 €","170,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,170000
2,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,95,2/6,P. kara,"2,495 €","237,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,237000
3,"Samazināta cena, iegūsti savu Laimi. \r\n\r\nP...",Valdemāra 123,3,102,6/6,P. kara,"1,372 €","139,900 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,139900
4,В доме всего 6 квартир. Печное отопление (комм...,Pērnavas 43,3,54,3/6,P. kara,950 €,"51,300 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,51300


In [43]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2669 entries, 0 to 2668
Data columns (total 11 columns):
Sludinājumi    2669 non-null object
Iela           2669 non-null object
Ist.           2669 non-null object
m2             2669 non-null int64
Stāvs          2669 non-null object
Sērija         2669 non-null object
Cena, m2       2669 non-null object
Cena           2669 non-null object
URL            2669 non-null object
Region         2669 non-null object
Price          2669 non-null int64
dtypes: int64(2), object(9)
memory usage: 229.4+ KB


In [44]:
df2["footage"] = df2["Cena, m2"].apply(lambda el: int(el.strip().split()[0].replace(",","")))
df2.head()

,Sludinājumi,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena,URL,Region,Price,footage
0,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,96,5/6,P. kara,"3,385 €","325,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,325000,3385
1,"Pārdod 3-istabu dzīvokli Rīgas klusajā centrā,...",Ausekļa 1,3,74,3/5,P. kara,"2,297 €","170,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,170000,2297
2,Nams būvēts 1907. gadā pēc arhitekta Aleksandr...,Barona 30,3,95,2/6,P. kara,"2,495 €","237,000 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,237000,2495
3,"Samazināta cena, iegūsti savu Laimi. \r\n\r\nP...",Valdemāra 123,3,102,6/6,P. kara,"1,372 €","139,900 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,139900,1372
4,В доме всего 6 квартир. Печное отопление (комм...,Pērnavas 43,3,54,3/6,P. kara,950 €,"51,300 €",https://www.ss.com/msg/lv/real-estate/flats/ri...,centre,51300,950


In [47]:
stats = df2.groupby(["Region"]).describe()

In [48]:
type(stats)

pandas.core.frame.DataFrame

In [49]:
stats

m2                                               \
                         count        mean        std    min     25%    50%   
Region                                                                        
agenskalns               146.0   69.102740  42.820512   17.0   43.25   56.0   
aplokciems                 9.0   52.000000  17.306068   27.0   46.00   55.0   
bergi                      4.0  102.750000  74.616240   25.0   49.75   99.0   
bierini                   11.0   42.636364  25.390048   21.0   24.00   37.0   
bolderaya                 34.0   50.823529  14.481902   25.0   41.50   50.0   
breksi                     1.0   66.000000        NaN   66.0   66.00   66.0   
bukulti                    1.0  148.000000        NaN  148.0  148.00  148.0   
bulli                      1.0  159.000000        NaN  159.0  159.00  159.0   
centre                   757.0   78.178336  45.643705   14.0   47.00   68.0   
chiekurkalns              27.0   43.666667  19.668405   14.0   31.00   40.0   
darzciems                 32.0   49.843750  13.447891   28.0   41.25   47.0   
daugavgriva               20.0   56.150000  13.677777   32.0   49.00   54.5   
dreilini                   2.0   60.000000  39.597980   32.0   46.00   60.0   
dzeguzhkalns              28.0   56.000000  16.931233   18.0   46.50   53.0   
grizinkalns               17.0   54.235294  22.221975   30.0   37.00   48.0   
ilguciems                 75.0   50.600000  21.613309   17.0   39.00   47.0   
imanta                   152.0   63.236842  23.766451   28.0   50.00   55.5   
jaunciems                  3.0   27.666667   6.350853   24.0   24.00   24.0   
jaunmilgravis              3.0   33.666667   4.041452   29.0   32.50   36.0   
katlakalns                 3.0   38.333333  23.459184   12.0   29.00   46.0   
kengarags                163.0   47.386503  11.290401   12.0   39.00   47.0   
kipsala                    8.0   92.000000  29.592470   46.0   75.00   84.5   
kleisti                    2.0   73.000000   7.071068   68.0   70.50   73.0   
kliversala                11.0   66.181818  21.908072   28.0   51.50   62.0   
krasta-st-area            21.0   57.380952  15.714567   28.0   49.00   56.0   
kundzinsala                1.0   55.000000        NaN   55.0   55.00   55.0   
mangali                   15.0   48.600000   9.053807   23.0   46.00   49.0   
mangalsala                 1.0   53.000000        NaN   53.0   53.00   53.0   
maskavas-priekshpilseta   35.0   40.457143  13.108917   18.0   29.50   39.0   
mezhapark                 41.0   71.975610  42.148836   14.0   43.00   67.0   
mezhciems                 78.0   68.153846  20.311904   32.0   52.50   64.0   
plyavnieki               131.0   57.465649  16.390568   18.0   45.50   58.0   
purvciems                244.0   61.254098  19.849873   15.0   48.75   59.0   
sarkandaugava             58.0   50.051724  53.858952   15.0   29.00   42.0   
shampeteris-pleskodale    36.0   79.194444  42.821770   18.0   52.00   71.0   
shkirotava                 9.0   60.888889  20.447765   44.0   49.00   52.0   
teika                     82.0   67.109756  37.062845   18.0   41.25   57.5   
tornjakalns               18.0   56.055556  51.200267   17.0   29.25   45.0   
vecaki                     1.0  171.000000        NaN  171.0  171.00  171.0   
vecmilgravis              86.0   53.313953  20.600549   18.0   42.25   51.0   
vecriga                   60.0   77.000000  59.856608   12.0   30.50   59.0   
vef                        2.0   77.500000  24.748737   60.0   68.75   77.5   
voleri                     2.0   33.000000   7.071068   28.0   30.50   33.0   
yugla                     67.0   66.268657  44.645602   22.0   42.50   47.0   
zasulauks                  4.0   26.500000   8.582929   17.0   20.75   26.5   
ziepniekkalns            115.0   63.721739  31.093675   13.0   40.50   61.0   
zolitude                  52.0   82.346154  30.403744   39.0   63.25   76.0   

                                        Price                 ...            

In [57]:
deals = stats.sort_values(by=[('Price','mean')], ascending=False)
deals

m2                                               \
                         count        mean        std    min     25%    50%   
Region                                                                        
kipsala                    8.0   92.000000  29.592470   46.0   75.00   84.5   
bukulti                    1.0  148.000000        NaN  148.0  148.00  148.0   
vecriga                   60.0   77.000000  59.856608   12.0   30.50   59.0   
vecaki                     1.0  171.000000        NaN  171.0  171.00  171.0   
centre                   757.0   78.178336  45.643705   14.0   47.00   68.0   
kliversala                11.0   66.181818  21.908072   28.0   51.50   62.0   
shampeteris-pleskodale    36.0   79.194444  42.821770   18.0   52.00   71.0   
mezhapark                 41.0   71.975610  42.148836   14.0   43.00   67.0   
kleisti                    2.0   73.000000   7.071068   68.0   70.50   73.0   
bulli                      1.0  159.000000        NaN  159.0  159.00  159.0   
zolitude                  52.0   82.346154  30.403744   39.0   63.25   76.0   
agenskalns               146.0   69.102740  42.820512   17.0   43.25   56.0   
teika                     82.0   67.109756  37.062845   18.0   41.25   57.5   
dreilini                   2.0   60.000000  39.597980   32.0   46.00   60.0   
mezhciems                 78.0   68.153846  20.311904   32.0   52.50   64.0   
imanta                   152.0   63.236842  23.766451   28.0   50.00   55.5   
bergi                      4.0  102.750000  74.616240   25.0   49.75   99.0   
aplokciems                 9.0   52.000000  17.306068   27.0   46.00   55.0   
grizinkalns               17.0   54.235294  22.221975   30.0   37.00   48.0   
purvciems                244.0   61.254098  19.849873   15.0   48.75   59.0   
ziepniekkalns            115.0   63.721739  31.093675   13.0   40.50   61.0   
dzeguzhkalns              28.0   56.000000  16.931233   18.0   46.50   53.0   
yugla                     67.0   66.268657  44.645602   22.0   42.50   47.0   
plyavnieki               131.0   57.465649  16.390568   18.0   45.50   58.0   
krasta-st-area            21.0   57.380952  15.714567   28.0   49.00   56.0   
shkirotava                 9.0   60.888889  20.447765   44.0   49.00   52.0   
darzciems                 32.0   49.843750  13.447891   28.0   41.25   47.0   
bierini                   11.0   42.636364  25.390048   21.0   24.00   37.0   
chiekurkalns              27.0   43.666667  19.668405   14.0   31.00   40.0   
ilguciems                 75.0   50.600000  21.613309   17.0   39.00   47.0   
katlakalns                 3.0   38.333333  23.459184   12.0   29.00   46.0   
breksi                     1.0   66.000000        NaN   66.0   66.00   66.0   
tornjakalns               18.0   56.055556  51.200267   17.0   29.25   45.0   
mangali                   15.0   48.600000   9.053807   23.0   46.00   49.0   
vef                        2.0   77.500000  24.748737   60.0   68.75   77.5   
kengarags                163.0   47.386503  11.290401   12.0   39.00   47.0   
maskavas-priekshpilseta   35.0   40.457143  13.108917   18.0   29.50   39.0   
mangalsala                 1.0   53.000000        NaN   53.0   53.00   53.0   
sarkandaugava             58.0   50.051724  53.858952   15.0   29.00   42.0   
vecmilgravis              86.0   53.313953  20.600549   18.0   42.25   51.0   
kundzinsala                1.0   55.000000        NaN   55.0   55.00   55.0   
daugavgriva               20.0   56.150000  13.677777   32.0   49.00   54.5   
bolderaya                 34.0   50.823529  14.481902   25.0   41.50   50.0   
zasulauks                  4.0   26.500000   8.582929   17.0   20.75   26.5   
voleri                     2.0   33.000000   7.071068   28.0   30.50   33.0   
jaunmilgravis              3.0   33.666667   4.041452   29.0   32.50   36.0   
jaunciems                  3.0   27.666667   6.350853   24.0   24.00   24.0   

                                        Price                 ...            

In [60]:
deals[('Price','mean')].values

array([234925.        , 210000.        , 160462.33333333, 150000.        ,
       136861.90488771, 120072.72727273, 117437.5       , 104127.80487805,
       102500.        ,  98000.        ,  93741.25      ,  91434.75342466,
        87797.36585366,  85750.        ,  80871.98717949,  80488.50657895,
        79750.        ,  77788.88888889,  72990.        ,  72130.99180328,
        71253.99130435,  68774.03571429,  67156.26865672,  58573.47328244,
        56048.57142857,  51665.55555556,  50849.96875   ,  48536.36363636,
        48073.66666667,  48073.06666667,  47370.        ,  45000.        ,
        44924.44444444,  44672.46666667,  43500.        ,  43489.98159509,
        41710.51428571,  41500.        ,  40619.12068966,  39437.02325581,
        38000.        ,  34228.        ,  32467.17647059,  18625.        ,
        16800.        ,  13166.66666667,  12683.33333333])